In [1]:
import main
main.create_csv_from_las()

Convert224ST4.LAS
Convert238ST4.LAS


Convert274ST4.LAS
Convert300.LAS


Convert301.LAS
Convert302.LAS


Convert304.LAS


Convert305.LAS
Convert306.LAS


...

Convert841.LAS
Convert850.LAS


Convert852.LAS
Convert853.LAS


Convert855.LAS
Convert856.LAS


Convert857.LAS
Convert859.LAS


end save csv


In [87]:
import numpy as np
import pandas as pd
import main

las_data = pd.read_csv(main.csv_out_file, delimiter=';')
water_content_data = pd.read_csv(main.data_dir + "\\res.csv", delimiter=';')

water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'wc'})
del water_content_data['Дата запуска']

result = pd.merge(las_data, water_content_data, on=main.keys_dict[main.kid_well])
print(len(result['well_name'].value_counts()))

173


In [54]:
result.head()

,well_name,STRT,STOP,DEPT,aps,rp,kp,kgl,kpr,kvo,kng,lit,satur,wc
0,300,3114.0,3363.6,3114.0,0.03,8.2,0.114,0.236,0.1,1.0,0.0,0.0,0.0,0.12
1,300,3114.0,3363.6,3114.1,0.03,8.2,0.114,0.236,0.1,1.0,0.0,0.0,0.0,0.12
2,300,3114.0,3363.6,3114.2,0.03,8.2,0.114,0.236,0.1,1.0,0.0,0.0,0.0,0.12
3,300,3114.0,3363.6,3114.3,0.03,8.2,0.114,0.236,0.1,1.0,0.0,0.0,0.0,0.12
4,300,3114.0,3363.6,3114.4,0.03,8.2,0.114,0.236,0.1,1.0,0.0,0.0,0.0,0.12


In [96]:
step_h_count = 50
# Выделяем даные для одной скважины
data_well = result[result['well_name'] == '300']

# высчитываем для нее характерисктики шага по глубине
start_h = data_well[:1]["STRT"]
end_h = data_well[:1]["STOP"]
start_h = float(start_h)
end_h = float(end_h)
h_step = (float(end_h) - float(start_h))/step_h_count
print(float(start_h), float(end_h), h_step)

3114.0 3363.6 4.991999999999998


In [104]:
h = start_h
temp_end = h + step_h_count * h_step
# выделяем для конкретного участка по глубине
data_well_by_h = data_well[(data_well['DEPT'] >= start_h) & (data_well['DEPT'] < end_h)]
# расчитываем среднее
data_well_by_h.pivot_table(['aps', 'rp', 'kp', 'kgl', 'kpr', 'kvo', 'kng', 'lit', 'satur'], ['well_name'] ,aggfunc='mean').head(10)

,aps,kgl,kng,kp,kpr,kvo,lit,rp,satur
well_name,,,,,,,,,
300,0.261222,0.249766,0.020361,0.09994,2.041629,0.952278,0.104167,18.075113,0.450321
